1.  **Install necessary packages:** Install libraries for transformers, sentence embeddings, FAISS for vector search, and document processing.

In [ ]:
!pip install transformers torch python-docx faiss-cpu sentence-transformers unsloth nltk scikit-learn pandas

2.  **Load and process the document:** Load the DOCX file and extract the text content.

In [ ]:
import os
import torch
import docx
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Path to the DOCX file
docx_path = "/content/Guilan-Food.docx"

# Check if the file exists
if not os.path.exists(docx_path):
    raise FileNotFoundError(f"DOCX file not found at {docx_path}. Please upload it.")

# Extract text from the DOCX file
print("Extracting text from DOCX...")
doc = docx.Document(docx_path)
text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
print(f"Extracted {len(text)} characters.")

3.  **Chunk the text:** Split the extracted text into smaller, overlapping chunks to facilitate embedding and retrieval.

In [ ]:
def chunk_text(text, chunk_size=150, overlap=20):
    """
    Splits text into overlapping chunks.
    - chunk_size: number of words per chunk
    - overlap: number of words to overlap between chunks
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap
    return chunks

# Create chunks from the extracted text
all_chunks = chunk_text(text, chunk_size=250, overlap=200)
print(f"Created {len(all_chunks)} text chunks.")

4.  **Generate embeddings:** Use a pre-trained sentence transformer model to create numerical representations (embeddings) of the text chunks.

In [ ]:
print("Generating embeddings for chunks...")
# Load a pre-trained sentence transformer model for multilingual embeddings
embedding_model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
# Generate embeddings for all text chunks
chunk_embeddings = embedding_model.encode(all_chunks, convert_to_numpy=True)

# Normalize the embeddings
chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings, axis=1).reshape(-1, 1)
print(f"Generated embeddings of shape: {chunk_embeddings.shape}")

5.  **Build FAISS index:** Create a FAISS index to enable fast and efficient similarity search over the chunk embeddings.

In [ ]:
# Get the dimension of the embeddings
dimension = chunk_embeddings.shape[1]
# Create a FAISS index for efficient similarity search using L2 distance
index = faiss.IndexFlatL2(dimension)  # L2 distance
# Add the chunk embeddings to the index
index.add(chunk_embeddings)
print("FAISS index built and embeddings added.")

6.  **Define retrieval function:** Create a function to retrieve the most relevant text chunks based on a given query using the FAISS index.

In [ ]:
def retrieve_contexts(query, k=3):
    """
    Retrieve the top-k most relevant text chunks for a query.
    """
    # Encode the query into an embedding
    query_embedding = embedding_model.encode([query])
    # Normalize the query embedding
    query_embedding = query_embedding / np.linalg.norm(query_embedding)  # normalize
    # Search the FAISS index for the k most similar chunks
    distances, indices = index.search(query_embedding, k)
    # Return the text of the top-k chunks
    return [all_chunks[i] for i in indices[0]]

7.  **Load Language Model:** Load a pre-trained causal language model (LLM) for generating answers. This example uses a quantized Llama-3 model for efficiency.

In [ ]:
# Define the name of the language model to load
model_name = "unsloth/llama-3-8b-bnb-4bit"

print(f"Loading model: {model_name}")
# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the causal language model with 4-bit quantization for efficiency
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # Automatically detect and use available devices (like GPU)
    torch_dtype=torch.float16 # Use float16 for reduced memory usage and faster computation
)
print("Model loaded successfully.")

8.  **Define RAG generation function:** Create a function that combines retrieval and generation. It retrieves relevant contexts for a question and then uses the LLM to generate an answer based on those contexts.

In [ ]:
def rag_generate(question, max_new_tokens=300):
    """
    Generate an answer using RAG:
    1. Retrieve relevant context
    2. Inject context into prompt
    3. Generate response with LLM
    """
    # Retrieve the top-k most relevant contexts for the question
    contexts = retrieve_contexts(question, k=3)
    # Format the retrieved contexts for the prompt
    context_str = "\n\n".join([f"[Reference {i+1}]: {ctx}" for i, ctx in enumerate(contexts)])

    # Define the chat template explicitly for Llama-3 to ensure correct formatting
    tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'system' %}{{ '<|start_header_id|>system<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}{% elif message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}{% elif message['role'] == 'assistant' %}{{ '<|start_header_id|>assistant<|end_header_id|>\n' + message['content'] + '<|eot_id|>' }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n' }}{% endif %}"

    # Create the messages list for the chat template
    messages = [
        {"role": "user", "content": f"first of all,You should answer in persian. Based on the following documents:\n{context_str}\n\nQuestion: {question}\nAnswer concisely."}
    ]
    # Apply the chat template to format the prompt
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Tokenize the prompt and move it to the appropriate device (GPU)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=3500).to(model.device)

    # Generate a response using the loaded language model
    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens, # Set the maximum number of new tokens to generate
            temperature=0.7, # Control the randomness of the output
            top_p=0.9, # Control the diversity of the output
            do_sample=True, # Enable sampling
            pad_token_id=tokenizer.eos_token_id # Set the padding token to the end-of-sequence token
        )

    # Decode the generated output
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the answer from the full response
    answer = full_response.split("[/INST]")[-1].strip()
    return answer

9.  **Test with example questions:** Use the RAG generation function to answer some example questions.

In [ ]:
# List of example questions in Persian
questions = [
    "طرز تهیه باقلا قاتوق چیست؟",
    "مواد لازم برای میرزا قاسمی چیه؟"
]

# Generate and print answers for each question using the RAG model
for q in questions:
    print("\n" + "="*100)
    print(f"Question: {q}")
    answer = rag_generate(q)
    print(f"Answer: {answer}")

10. **Advanced Evaluation (Optional):** This section includes code for fine-tuning the embedding model and evaluating the RAG system using metrics like MRR, F1 score, and Cosine Similarity. This is more advanced and may require additional setup and data.

In [ ]:
# Install required packages
!pip install -q transformers==4.55.2 sentence-transformers datasets torch bitsandbytes python-docx nltk
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import unsloth  # Import unsloth before transformers
import re
import numpy as np
import nltk
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
from docx import Document
from sentence_transformers import SentenceTransformer, losses, util
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from datasets import Dataset
import torch
from unsloth import FastLanguageModel

# Free GPU memory
torch.cuda.empty_cache()

# Load DOCX content
try:
    doc = Document('Guilan-Food.docx')
    text = '\n'.join([para.text for para in doc.paragraphs])
    # Clean up text by replacing special characters and reducing multiple spaces
    text = re.sub(r'\·|\•', '-', text)
    text = re.sub(r'\s+', ' ', text).strip()
except FileNotFoundError:
    raise FileNotFoundError("Please upload 'Guilan-Food.docx' to the Colab environment.")

# Questions and their corresponding ground truth answers
questions = [
    "مواد لازم برای باقلا قاتوق برای 4 نفر چیست؟",
    "طرز تهیه میرزا قاسمی چگونه است؟",
    "مواد لازم برای کباب ترش برای 4 نفر چیست؟",
    "طرز تهیه رشته خشکار چگونه است؟",
    "مواد لازم برای اناربیج برای 4 نفر چیست؟"
]
ground_truths = [
    "باقلای کشاورزی: 500 گرم (پوست کنده شده) - شوید تازه: 200 گرم (ریز خرد شده) - سیر: 3-4 حبه (رنده یا له شده) - تخم مرغ: 2-3 عدد - روغن مایع یا کره: به مقدار لازم - نمک، فلفل سیاه و زردچوبه: به مقدار لازم - آب: حدود 1 لیوان",
    "برای تهیه میرزا قاسمی، ابتدا بادمجان‌ها را بشویید و روی شعله گاز، منقل یا داخل فر کبابی کنید تا پوست آن‌ها کاملاً بسوزد و داخلشان نرم شود. در حین کباب کردن، بادمجان‌ها را بچرخانید تا تمام قسمت‌ها به خوبی کباب شوند. سپس بگذارید بادمجان‌ها کمی خنک شوند و پوست سوخته آن‌ها را جدا کنید. گوشت داخل بادمجان‌ها را با چاقو ساطوری کنید، نه خیلی ریز و نه خیلی درشت. در مرحله بعد، گوجه فرنگی‌ها را بشویید و پوست آن‌ها را بگیرید. برای راحت‌تر پوست گرفتن، می‌توانید گوجه‌ها را چند ثانیه در آب جوش قرار دهید. سپس گوجه فرنگی‌ها را نگینی خرد کنید یا رنده کنید. در یک تابه مناسب، مقداری روغن یا کره بریزید و سیر له شده یا رنده شده را اضافه کنید. کمی تفت دهید تا عطر سیر بلند شود، مراقب باشید که نسوزد. سپس گوجه فرنگی‌های خرد شده یا رنده شده را به تابه اضافه کنید و تفت دهید تا آب آن کشیده شود و کمی غلیظ شود. در این مرحله، کمی نمک، فلفل سیاه و زردچوبه به آن اضافه کنید. حالا بادمجان ساطوری شده را به تابه اضافه کنید و با گوجه فرنگی و سیر مخلوط کنید. حدود 5-10 دقیقه تفت دهید تا طعم‌ها به خوبی ترکیب شوند. سپس در وسط تابه یک گودی ایجاد کرده و تخم‌مرغ‌ها را در آن بشکنید. اجازه دهید تخم‌مرغ‌ها کمی بپزند و خودشان را بگیرند، سپس آن‌ها را با مواد دیگر مخلوط کرده و هم بزنید تا کاملاً پخته و یکدست شوند. در نهایت، اجازه دهید میرزا قاسمی چند دقیقه دیگر روی حرارت ملایم بماند تا روغن بیندازد و طعم‌ها کاملاً جا بیفتند. سپس آن را در ظرف مناسب بریزید و در صورت تمایل با کمی جعفری خرد شده تزئین کنید.",
    "گوشت گوساله یا گوسفند: 500 گرم (تکه شده برای کباب، معمولاً از راسته یا فیله استفاده می‌شود) - گردوی آسیاب شده: 100 گرم - رب انار ترش یا ملس: 3-4 قاشق غذاخوری (بسته به غلظت و ترشی رب) - آب انار ترش: 2-3 قاشق غذاخوری (اختیاری، برای طعم بیشتر) - سیر: 2-3 حبه (رنده یا له شده) - پیاز متوسط: 1 عدد (رنده شده و آب آن گرفته شده) - سبزیجات معطر محلی (چوچاق و خالواش) تازه یا خشک: 2-3 قاشق غذاخاری (اگر در دسترس نبود، می‌توانید از ترکیب گشنیز، جعفری و نعناع به مقدار کم استفاده کنید) - روغن زیتون یا روغن مایع: 2-3 قاشق غذاخوری - نمک و فلفل سیاه: به مقدار لازم",
    "برای تهیه رشته خشکار ابتدا آرد برنج را در یک تابه با کمی کره تفت دهید تا کمی رنگ آن تغییر کند. سپس شیر را به آرد برنج اضافه کنید و هم بزنید تا مخلوطی یکدست و بدون گلوله به دست آید. حالا شکر و پودر هل را اضافه کرده و به هم زدن ادامه دهید تا شکر کاملاً حل شود. بعد از این مرحله، آب و گلاب را به مخلوط اضافه کنید و اجازه دهید مواد روی حرارت ملایم به جوش بیاید. هنگامی که مایع غلیظ شد، آن را در قالب‌های مناسب بریزید و بگذارید کمی سرد شود. در نهایت رشته خشکار را با پودر نارگیل تزئین کنید و در صورت تمایل از پسته یا بادام خرد شده نیز استفاده کنید.",
    "گوشت چرخ‌کرده: 400 گرم - گردوی آسیاب شده: 200 گرم - رب انار ترش یا ملس: 3 قاشق غذاخوری - سبزی معطر (چوچاق، خالواش، گشنیز، نعناع): 200 گرم - پیاز متوسط: 1 عدد (رنده شده) - آب: حدود 3 لیوان - نمک، فلفل، زردچوبه: به مقدار لازم"
]

# Chunking function
def chunk_text(text, chunk_size=150, overlap=20):
    """
    Splits text into overlapping chunks.
    - chunk_size: number of words per chunk
    - overlap: number of words to overlap between chunks
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap
    return chunks

# Create chunks
all_chunks = chunk_text(text, chunk_size=150, overlap=20)
print(f"Created {len(all_chunks)} text chunks.")

# F1 score function for evaluating generated responses
def compute_f1(pred, gt):
    pred_tokens = set(pred.split())
    gt_tokens = set(gt.split())
    common = pred_tokens & gt_tokens
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = len(common) / len(gt_tokens) if len(gt_tokens) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1

# Function to get relevant chunk indices based on ground truth text
def get_relevant_chunks(chunks, gt, threshold=0.5):
    gt_tokens = set(gt.split())
    relevant = []
    for i, chunk in enumerate(chunks):
        chunk_tokens = set(chunk.split())
        # Calculate overlap as a ratio
        overlap = len(gt_tokens & chunk_tokens) / len(gt_tokens) if len(gt_tokens) > 0 else 0
        if overlap > threshold:
            relevant.append(i)
    return relevant

# Fine-tune embedding model with more relevant training data
sentences = nltk.sent_tokenize(text)
train_data = []
# Create positive pairs (consecutive sentences)
for i in range(len(sentences) - 1):
    train_data.append({"text1": sentences[i], "text2": sentences[i + 1], "label": 1.0})
# Create negative pairs (sentences further apart) - adjusted logic for creating negative pairs
# A simple way is to pair sentences that are not consecutive.
# This implementation creates pairs of sentences that are two steps apart.
for i in range(len(sentences) - 2):
     train_data.append({"text1": sentences[i], "text2": sentences[i + 2], "label": 0.0}) # Use 0.0 for non-related pairs for ContrastiveLoss

# Create a Dataset from the training data
train_dataset = Dataset.from_list(train_data)

# Load the pre-trained embedding model
embedding_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
# Define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="fine_tuned_model",
    num_train_epochs=2,  # Increased for better fine-tuning
    per_device_train_batch_size=4,
    warmup_steps=5,
    fp16=True,
    logging_steps=10,
)
# Define the loss function (ContrastiveLoss is suitable for positive/negative pairs)
train_loss = losses.ContrastiveLoss(embedding_model)
# Initialize the trainer
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss
)
# Start fine-tuning
trainer.train()

# Save the fine-tuned model
embedding_model.save("fine_tuned_model")

# Load fine-tuned model, with fallback to original if loading fails
try:
    embedding_model = SentenceTransformer("fine_tuned_model")
except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    print("Falling back to the original model.")
    embedding_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

# Free GPU memory
torch.cuda.empty_cache()

# Load LLaMA model with reduced sequence length for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=1024,
    load_in_4bit=True,
)
# Prepare the model for inference
FastLanguageModel.for_inference(model)

# Function to retrieve top-k chunks based on query similarity
def get_top_k_chunks(query, chunks, model, k=10):  # Increased k for more retrieval
    if not chunks:
        return [], []
    # Encode the query and all chunks
    query_embedding = model.encode(query, batch_size=1)
    chunk_embeddings = model.encode(chunks, batch_size=1)
    # Calculate cosine similarity between the query and each chunk
    similarities = util.cos_sim(query_embedding, chunk_embeddings)[0]
    if len(similarities) == 0:
        return [], []
    # Get the indices of the top-k most similar chunks
    top_k_indices = np.argsort(similarities.numpy())[::-1][:k]
    # Return the top-k chunks and their indices
    return [chunks[i] for i in top_k_indices], top_k_indices

# Response generation function
def generate_response(query, context):
    # Define the prompt template for the language model
    prompt = f"""Based on the following context, only answer the question in Persian.
Do not repeat the question or give explanations.
Do not ask the question.
Only return the direct answer.

Context: {context[:2000]}

Question: {query}

Answer:"""
    # Tokenize the prompt and move to GPU
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # Generate the response using the language model
    outputs = model.generate(**inputs, max_new_tokens=200, eos_token_id=tokenizer.eos_token_id)  # Reduced max_new_tokens
    # Decode the generated output
    decoded = tokenizer.decode(outputs[0])
    # Extract the answer part based on the "Answer:" marker
    if "Answer:" in decoded:
        response = decoded.split("Answer:")[-1].strip()
    else:
        response = decoded.strip()
    # Clean up the response
    response = response.replace("<|end_of_text|>", "").strip()
    # Print query and response for monitoring
    print("===="*40)
    print(f"Query: {query}")
    print(f"Response: {response}")
    # Free GPU memory after generation
    torch.cuda.empty_cache()
    return response

# Evaluation loop
k = 10  # Increased k to retrieve more relevant chunks
f1_scores = []
cosine_scores = []
mrr_scores = []

print(f"\n----------------------")
print(f"| Overlap Chunking: |")
print(f"----------------------")

# Iterate through each question and its ground truth
for idx, (query, gt) in enumerate(zip(questions, ground_truths)):
    # Retrieve top-k chunks
    top_chunks, top_indices = get_top_k_chunks(query, all_chunks, embedding_model, k=k)
    if not top_chunks:
        print(f"No chunks retrieved for query: {query}")
        # Append 0 scores if no chunks are retrieved
        f1_scores.append(0.0)
        cosine_scores.append(0.0)
        mrr_scores.append(0.0)
        continue
    # Combine retrieved chunks into a single context string
    context = '\n\n'.join(top_chunks)
    # Generate response using the language model with the retrieved context
    response = generate_response(query, context)

    # Generation evaluation (F1 and Cosine Similarity)
    f1 = compute_f1(response, gt)
    # Encode the generated response and ground truth for cosine similarity calculation
    emb_response = embedding_model.encode(response, batch_size=1)
    emb_gt = embedding_model.encode(gt, batch_size=1)
    cosine = util.cos_sim(emb_response, emb_gt)[0][0].item()

    # Retrieval evaluation (MRR)
    # Get the indices of truly relevant chunks based on the ground truth
    relevant = get_relevant_chunks(all_chunks, gt)
    mrr = 0
    # Calculate MRR based on the rank of the first relevant retrieved chunk
    for rank, idx in enumerate(top_indices, 1):
        if idx in relevant:
            mrr = 1 / rank
            break

    # Append calculated scores
    f1_scores.append(f1)
    cosine_scores.append(cosine)
    mrr_scores.append(mrr)

    # Free GPU memory after each iteration
    torch.cuda.empty_cache()

# Print final average results
print("\nResults:")
print(f"F1: {np.mean(f1_scores):.4f}")
print(f"Cosine Similarity: {np.mean(cosine_scores):.4f}")
print(f"MRR: {np.mean(mrr_scores):.4f}")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Created 34 text chunks.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 179 | Num Epochs = 1 | Total steps = 45
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 135,127,808 of 135,127,808 (100.00% trained)


Step,Training Loss
10,1.729500
20,2.147600
30,1.614600
40,1.472300


==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

----------------------
| Overlap Chunking: |
----------------------
Query: مواد لازم برای باقلا قاتوق برای 4 نفر چیست؟
Response: برای تهیه رشته خشکار، ابتدا آرد برنج را در یک تابه با کمی کره تفت دهید تا کمی رنگ آن تغییر کند. سپس شیر را به آرد برنج اضافه کنید و هم بزنید تا مخلوطی یکدست و بدون گل
Query: طرز تهیه میرزا قاسمی چگونه است؟
Response: برای تهیه میرزا قاسمی، ابتدا بادمجان‌ها را بشویید و روی شعله گاز، منقل یا داخل فر کبابی کنید تا پوست آن‌ها کاملاً بسوزد و داخلشان نرم شود. در حین کباب کردن، بادمجان‌ها را بچرخانید تا تمام قسمت‌ها به خوبی کب